In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Data
$y=ax+b$

In [ ]:
f = lambda x: x*2.0 + 1.0

xs = np.random.rand(1000, 1)   # 1000 points
ys = f(xs) + 0.1*np.random.randn(1000, 1)

plt.title("Dataset")
plt.scatter(xs, ys, s=1)
plt.plot(xs, f(xs), label=f"y = 2x + 1")
plt.legend()
plt.show()

# Model

* A model is a mathematical function $f_{\theta}:\mathbb R\rightarrow \mathbb R$ with some parameters $\theta=\{a, b\}$.
* We need:
  * a function $f$,
  * a way to set and get parameters,
  * a way to process a bunch (called a batch) of inputs.

In [3]:
class Module:
  def __init__(self) -> None:
    raise NotImplementedError

  def set_params(self) -> None:
    raise NotImplementedError
  
  def get_params(self) -> dict:
    raise NotImplementedError
  
  def forward(self, x:np.array) -> np.array:
    raise NotImplementedError
  
  def __call__(self, x:np.array) -> np.array:
    return self.forward(x)

NameError: name 'np' is not defined

In [ ]:
class LinearModel(Module):
  def __init__(self, w:float=.0, b:float=.0) -> None:
    self.set_params(w, b)

  def set_params(self, w:float, b:float) -> None:
    self.w = w
    self.b = b

  def get_params(self) -> dict[str,float]:
    return {'w': self.w, 'b':self.b}

  def forward(self, x:np.array) -> np.array:
    params = self.get_params()
    w = params.get('w')
    b = params.get('b')
    return w * x + b

In [ ]:
def mse(y_hat:np.array, y_true:np.array) -> float:
  assert len(y_hat) == len(y_true)
  return ((y_hat - y_true)**2).mean()

In [ ]:
lin = LinearModel(2,1)

In [ ]:
mse(lin(xs), ys)

$f_\theta(x) = x w + b$ where $\theta=(w,b)$.

$\operatorname{MSE}_\theta(x, y) = \frac{1}{n}\sum^n_{i=1}(f(x_i) - y_i)^2 = \frac{1}{n}\sum^n_{i=1}(x_iw+b - y_i)^2$.

$$\frac{\partial}{\partial w} \operatorname{MSE}_{\theta}(x,y) = \frac{2}{n}\sum^n_{i=1}x_i(x_iw+b - y_i)=\frac{2}{n}\sum^n_{i=1}x_i(\hat y_i - y_i).$$

$$\frac{\partial}{\partial b} \operatorname{MSE}_{\theta}(x,y) = \frac{2}{n}\sum^n_{i=1}(x_iw+b - y_i)= \frac{2}{n}\sum^n_{i=1}(\hat y_i - y_i).$$

$f,g:\mathbb R\rightarrow \mathbb R$

- $f(x)=c \implies f'(x)=0$

- $f(x)=ax+b \implies f'(x)=a$

- $f(x)=ax^2 + bx + c \implies f'(x)=2ax + b$

- $(f(x) + g(x))' = f'(x) + g'(x)$

- $(f(x)g(x))' = f'(x)g(x) + f(x)g'(x)$

- $(f(g(x)))' = f'(g(x))g'(x)$

- $\frac{d(x^2)}{dx} = (x^2)' = 2x$

$\frac{\partial}{\partial w}(x_iw+b - y_i)^2 = \frac{\partial}{\partial w}(t(w))^2 = 2t(w)t'(w)$

where $t(w)=(x_iw+b - y_i)$, $t'(w)=x_i$.

Therefore, $\frac{\partial}{\partial w}(x_iw+b - y_i)^2 = 2x_i(x_iw+b - y_i)$.

$$\frac{\partial}{\partial w} \operatorname{MSE}_{\theta}(x,y) = \frac{2}{n}\sum^n_{i=1}x_i(x_iw+b - y_i)=\frac{2}{n}\sum^n_{i=1}x_i(\hat y_i - y_i).$$

$$\frac{\partial}{\partial b} \operatorname{MSE}_{\theta}(x,y) = \frac{2}{n}\sum^n_{i=1}(x_iw+b - y_i)= \frac{2}{n}\sum^n_{i=1}(\hat y_i - y_i).$$

In [ ]:
def grad_mse(model:Module, x:np.array, y_true:np.array) -> dict[str,float]:
  assert len(x) == len(y_true)
  n = len(x)
  y_hat = model(x)
  d_w = 2*(x*(y_hat-y_true)).mean()
  d_b = 2*(y_hat-y_true).mean()
  return {'d_w': d_w, 'd_b':d_b}

In [ ]:
lin2 = LinearModel(0,0)
grad_mse(lin2, xs, ys)

* Update $\theta=\{w,b\}$: $$\theta_{\textrm{new}} = \theta_{\textrm{old}} - \alpha \nabla\operatorname{MSE}_{\theta_{\textrm{old}}}(x,y)$$ where $\alpha>0$ is a learning rate.

In [ ]:
def update(model:Module, lr:float, d_w:float, d_b:float) -> None:
  params_old = model.get_params()
  params_new = {
    'w': params_old.get('w') - lr*d_w,
    'b': params_old.get('b') - lr*d_b,
  }
  model.set_params(**params_new)

In [ ]:
lin2.get_params()

In [ ]:
update(lin2, lr=0.001, **grad_mse(lin2, xs, ys))

In [ ]:
lin2.get_params()

In [ ]:
history = [lin2.get_params()]

for epoch in range(200):
  grad = grad_mse(lin2, xs, ys)
  update(lin2, 0.2, **grad)
  err = mse(lin2(xs), ys)
  params = lin2.get_params()
  history.append(params)
  print(f"Epoch {epoch+1}: mse={err:.4f}, w={params.get('w'):.4f}, b={params.get('b'):.4f}")

```bash
conda activate MathAI
pip install pandas plotly nbformat
```

In [ ]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame(history, columns=['w','b'])
df = df.set_index(df.index.set_names('epoch')).reset_index()
df0 = df.copy()
df1 = df.copy()
df0['x'] = xs.min()
df1['x'] = xs.max()
df = pd.concat([df0, df1]).reset_index(drop=True)
df['y'] = df.w * df.x + df.b

fig = px.line(df, x='x', y='y', animation_frame="epoch", width=500, height=500)

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 0.1
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 0.1
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['redraw'] = True

fig.add_scatter(x=xs.flatten(), y=ys.flatten(), mode='markers', name='data', marker={'size':2})

for i, frame in enumerate(fig.frames):
    frame['layout']['title_text'] = f"Prediction: y = {history[i]['w']:.4f}x{'' if history[i]['b'] < 0 else '+'}{history[i]['b']:.4f}"

fig.update_layout(template='plotly_dark')
fig.show()